# Toronto Analysis

In [197]:
# Conda
#!conda install -c conda-forge beautifulsoup4 --yes
#!conda install -c conda-forge lxml --yes
#!conda install -c conda-forge requests --yes
#!conda install -c conda-forge folium=0.5.0 --yes

In [249]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import json
from pandas.io.json import json_normalize
import numpy as np

##  Toronto Postal codes and Geodata

In [267]:
# Use work done in part 1 and 2

page_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_response = requests.get(page_link, timeout=5)
soup=BeautifulSoup(page_response.content, 'lxml')
table=soup.find('table', class_='wikitable sortable')
column_names=['Postcode', 'Borough', 'Neighbourhood']

nrows=0
for tr in table.find_all('tr'):
    column=tr.find_all('td')
    if len(column)>0:
        nrows+=1
          
df = pd.DataFrame(columns=column_names, index=range(0,nrows))

row_marker=0
for tr in table.find_all('tr'):
    column_marker=0
    column=tr.find_all('td')
    for td in column:
        df.iat[row_marker, column_marker]=td.get_text() 
        column_marker +=1
    if len(column)>0:
        row_marker +=1

df.replace(r'\s+|\\n', ' ', regex=True, inplace=True)
df=df[df.Borough != "Not assigned"]
df=df[df.Neighbourhood != "Not assigned"]
df=df.groupby('Postcode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
df_g=pd.read_csv("http://cocl.us/Geospatial_data")

neighborhoods = pd.merge(df, df_g,  left_on='Postcode', right_on='Postal Code')
neighborhoods=neighborhoods[["Postal Code", "Borough", "Neighbourhood", "Latitude", "Longitude"]]
neighborhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Highland Creek , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"West Hill , Morningside , Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Exploring Toronto

In [268]:
## Boroughs containing "Toronto" only
toronto_data=neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]
toronto_data=toronto_data[["Neighbourhood", "Latitude", "Longitude"]]
toronto_data.head()

,Neighbourhood,Latitude,Longitude
37,The Beaches,43.676357,-79.293031
41,"The Danforth West , Riverdale",43.679557,-79.352188
42,"The Beaches West , India Bazaar",43.668999,-79.315572
43,Studio District,43.659526,-79.340923
44,Lawrence Park,43.728020,-79.388790


In [330]:
# Toronto coordinates
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Explore Neighbourhoods

In [332]:
# Get venues from foursquare
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [333]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches 
The Danforth West , Riverdale 
The Beaches West , India Bazaar 
Studio District 
Lawrence Park 
Davisville North 
North Toronto West 
Davisville 
Summerhill East , Moore Park 
Rathnelly , Forest Hill SE , Deer Park , Summerhill West , South Hill 
Rosedale 
St. James Town , Cabbagetown 
Church and Wellesley 
Harbourfront , Regent Park 
Garden District , Ryerson 
St. James Town 
Berczy Park 
Central Bay Street 
King , Adelaide , Richmond 
Toronto Islands , Harbourfront East , Union Station 
Design Exchange , Toronto Dominion Centre 
Commerce Court , Victoria Hotel 
Roselawn 
Forest Hill West , Forest Hill North 
Yorkville , The Annex , North Midtown 
Harbord , University of Toronto 
Kensington Market , Grange Park , Chinatown 
South Niagara , Harbourfront West , Railway Lands , Bathurst Quay , Island airport , King and Spadina , CN Tower 
Stn A PO Boxes 25 The Esplanade 
Underground city , First Canadian Place 
Christie 
Dufferin , Dovercourt Village 
Little Portugal , Trini

In [334]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West , Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [335]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
1,Business Reply Mail Processing Centre 969 East...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.00,0.00000,0.000000,0.00,0.011364,0.000000,0.000000,0.011364,0.000000,0.011364
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.011364,0.011364,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.011364,...,0.00,0.00000,0.000000,0.00,0.000000,0.011364,0.011364,0.000000,0.011364,0.011364
5,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,...,0.00,0.00000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
6,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.03125,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Design Exchange , Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,...,0.00,0.00000,0.000000,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Dufferin , Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [336]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [337]:
# Show top venues pr neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Steakhouse,Beer Bar,Bakery
1,Business Reply Mail Processing Centre 969 East...,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Burger Joint,Chinese Restaurant,Middle Eastern Restaurant,Salad Place,Restaurant,Bubble Tea Shop,Bar
3,Christie,Grocery Store,Café,Park,Nightclub,Diner,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Convenience Store
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Nightclub,Bubble Tea Shop,Men's Store,Café,Mediterranean Restaurant
5,"Commerce Court , Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Gastropub,Bakery,Deli / Bodega,Steakhouse
6,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gourmet Shop,Dance Studio,Seafood Restaurant
7,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Clothing Store,Hotel,Discount Store,Dog Run,Doner Restaurant
8,"Design Exchange , Toronto Dominion Centre",Coffee Shop,Hotel,Café,Restaurant,Seafood Restaurant,Gastropub,Steakhouse,Deli / Bodega,Italian Restaurant,American Restaurant
9,"Dufferin , Dovercourt Village",Pharmacy,Supermarket,Bakery,Gym / Fitness Center,Fast Food Restaurant,Liquor Store,Middle Eastern Restaurant,Discount Store,Music Venue,Park


## Cluster Neighborhoods

In [338]:
# k means
kclusters = 5# set number of clusters
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [339]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() 

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,43.676357,-79.293031,3,Music Venue,Health Food Store,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
41,"The Danforth West , Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Furniture / Home Store,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
42,"The Beaches West , India Bazaar",43.668999,-79.315572,1,Park,Sandwich Place,Pet Store,Ice Cream Shop,Burger Joint,Liquor Store,Light Rail Station,Burrito Place,Fast Food Restaurant,Fish & Chips Shop
43,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Fish Market,Latin American Restaurant,Coworking Space
44,Lawrence Park,43.728020,-79.388790,1,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [340]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'], toronto_merged['Neighbourhood'],toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyze clusters

In [341]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0,] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Summerhill East , Moore Park",Playground,Tennis Court,Trail,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dumpling Restaurant
50,Rosedale,Park,Playground,Trail,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [342]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0,] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"The Danforth West , Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Furniture / Home Store,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
42,"The Beaches West , India Bazaar",Park,Sandwich Place,Pet Store,Ice Cream Shop,Burger Joint,Liquor Store,Light Rail Station,Burrito Place,Fast Food Restaurant,Fish & Chips Shop
43,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Cheese Shop,Fish Market,Latin American Restaurant,Coworking Space
44,Lawrence Park,Park,Construction & Landscaping,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
45,Davisville North,Gym,Food & Drink Shop,Sandwich Place,Park,Breakfast Spot,Clothing Store,Hotel,Discount Store,Dog Run,Doner Restaurant
46,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Fast Food Restaurant,Pet Store,Chinese Restaurant,Rental Car Location,Dessert Shop,Salon / Barbershop,Diner
47,Davisville,Sandwich Place,Pizza Place,Dessert Shop,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gourmet Shop,Dance Studio,Seafood Restaurant
49,"Rathnelly , Forest Hill SE , Deer Park , Summe...",Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,American Restaurant,Convenience Store,Light Rail Station
51,"St. James Town , Cabbagetown",Coffee Shop,Pub,Pizza Place,Restaurant,Café,Market,Italian Restaurant,Bakery,Playground,Liquor Store
52,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Nightclub,Bubble Tea Shop,Men's Store,Café,Mediterranean Restaurant


In [343]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0,] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselawn,Garden,Filipino Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [344]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0,] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,Music Venue,Health Food Store,Pub,Neighborhood,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [345]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0,] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,"Forest Hill West , Forest Hill North",Trail,Sushi Restaurant,Jewelry Store,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
